In [ ]:
%pip install -U weaviate-client

In [ ]:
%pip install pillow

In [ ]:
import weaviate
import os

client = weaviate.connect_to_local(
    host="weaviate",
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]  # Replace with your inference API key
    }
)

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

In [ ]:
class_name = "Fuga"
client.collections.create(
    name=class_name,
    description="",
    vectorizer_config=Configure.Vectorizer.img2vec_neural(
        image_fields=["image"],
    ),
    vector_index_config=Configure.VectorIndex.hnsw(),
    properties=[
        Property(name="image", data_type=DataType.BLOB),
        Property(name="text", data_type=DataType.TEXT)
    ],   
)
print("Schema successfully created..!!")

In [ ]:
from IPython.display import Image
import base64

In [ ]:
hoge_dir = "/home/jovyan/data/hoge"
for path in os.listdir(hoge_dir):
    display(Image(filename=f"{hoge_dir}/{path}"))

In [ ]:
# Upload the image to the database in batches 
batch_size = 10 # Set your preferred batch size
count = 0
collection = client.collections.get(class_name)

# Add data objects in batches to the client
with collection.batch.dynamic() as batch:
    for filename in os.listdir(hoge_dir):
        if filename.endswith('.png'):
            with open(f'{hoge_dir}/{filename}','rb') as img_file:
                img_data = img_file.read() 

                # convert image into base64
                base64_img = base64.b64encode(img_data).decode('utf-8') 
                text,_ = os.path.splitext(filename) 
                data = {
                    'image': base64_img,
                    'text': text
                }

                batch.add_object(
                    properties=data,
                )
                
                # Print progress for every batch_size images uploaded
                if count % batch_size == 0:
                    print(count)
                count +=1

print("Images are uploaded to the database Successfully...!!")

In [ ]:
test_img_path = "/home/jovyan/data/piyo"
for path in os.listdir(test_img_path):
    display(Image(filename=f"{test_img_path}/{path}"))
    print(path)

In [ ]:
from pathlib import Path
response = collection.query.near_image(
    near_image=Path(test_img_path + ""),
    return_properties=["text","image"],
    limit=3
)
# display(response.objects[0])

In [ ]:
# base64形式の文字列を画像として表示する
from PIL import Image as PILImage
from io import BytesIO

img_data = base64.b64decode(response.objects[0].properties["image"])
img = PILImage.open(BytesIO(img_data))
img.show()

In [ ]:
img_data = base64.b64decode(response.objects[1].properties["image"])
img = PILImage.open(BytesIO(img_data))
img.show()

In [ ]:
img_data = base64.b64decode(response.objects[2].properties["image"])
img = PILImage.open(BytesIO(img_data))
img.show()